In [1]:
import os
from dotenv import load_dotenv
load_dotenv(encoding='utf-8')

True

# RAG pipeline

## Vector Store and Retriever

In [2]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_mongodb import MongoDBAtlasVectorSearch
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from pymongo import MongoClient

In [3]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(model=os.getenv("DEFAULT_OPENAI_MODEL"))
embedding_model=OpenAIEmbeddings(disallowed_special=())

In [4]:
# Define MongoDB vector database
client = MongoClient(os.getenv("ATLAS_CONNECTION_STRING"))
db_name = os.getenv("db_name")
collection_name = "enterprise_data"
atlas_collection = client[db_name][collection_name]
index_name = "vector_index_erp"

vector_store = MongoDBAtlasVectorSearch(
    embedding = embedding_model,
    collection = atlas_collection,
    index_name = index_name
)

retriever = vector_store.as_retriever(
    search_type = "similarity",
    search_kwargs = { "k": 10}  # "score_threshold": 0.75 
)

## RAG with different prompt templates

### Base Template

In [5]:
# Define a prompt template
import pprint
def call_openai_template_1(question):

   question = question['question']

   base_template = """
   Use the following pieces of context to answer the question at the end.
   If you don't know the answer, just say that you don't know, don't try to make up an answer.

   {context}

   Question: {question}
   """
   custom_rag_prompt = PromptTemplate.from_template(base_template)

   #llm = get_llm_model("openai")
   llm = ChatOpenAI(model=os.getenv("DEFAULT_OPENAI_MODEL"))

   def format_docs(docs):
      return "\n\n".join(doc.page_content for doc in docs)

   # Construct a chain to answer questions on your data
   rag_chain = (
      { "context": retriever | format_docs, "question": RunnablePassthrough()}
      | custom_rag_prompt
      | llm
      | StrOutputParser()
   )

   # Prompt the chain
   question = question
   answer = rag_chain.invoke(question)
   similar = retriever.invoke(question)


   return{
      'answer': answer,
      'contexts': similar
      }

In [6]:
# Test sample
question = {'question': "How does a supportive culture impact employee engagement and align with Tech Innovators Inc.'s approach to employment relations and engagement?"}
answer = call_openai_template_1(question)
print(answer['answer'][:150])

A supportive culture positively impacts employee engagement by fostering an environment where employees feel valued, respected, and heard. This type o


In [7]:
print(answer['contexts'])

[Document(metadata={'_id': '66d8163d0533e009aa2d4b0f', 'pageid': '491566', 'department': 'HR', 'title': 'Employee Relations and Engagement at Tech Innovators Inc.'}, page_content='employees and management.Employee Involvement: Involving employees in decision-making processes and seeking their input on matters that affect them.Supportive Culture: Creating a supportive culture'), Document(metadata={'_id': '66d816590533e009aa2d4d34', 'pageid': '491566', 'department': 'HR', 'title': 'Employee Relations and Engagement at Tech Innovators Inc.'}, page_content='employees and management.Employee Involvement: Involving employees in decision-making processes and seeking their input on matters that affect them.Supportive Culture: Creating a supportive culture'), Document(metadata={'_id': '66d816750533e009aa2d4f59', 'pageid': '491566', 'department': 'HR', 'title': 'Employee Relations and Engagement at Tech Innovators Inc.'}, page_content='employees and management.Employee Involvement: Involving emp

### Prompt Template 2: Who are you template

In [22]:
# Define a prompt template
import pprint
def call_openai_template_2(question):

   question = question['question']

   template_2 = """
    "You are an enterprise Chatbot, an AI assistant designed to retrieve information from the enterprise Confluence system. 
    You specialize in providing accurate answers related to various departments like Marketing, IT, HR, Finance, and Corporate Communications. 

    Use the following pieces of context to answer the question at the end.
    If you don't know the answer, just say that you don't know, don't try to make up an answer.

    {context}

    Question: {question}
   """

   custom_rag_prompt = PromptTemplate.from_template(template_2)

   llm = ChatOpenAI(model=os.getenv("DEFAULT_OPENAI_MODEL"))

   def format_docs(docs):
      return "\n\n".join(doc.page_content for doc in docs)

   # Construct a chain to answer questions on your data
   rag_chain = (
      { "context": retriever | format_docs, "question": RunnablePassthrough()}
      | custom_rag_prompt
      | llm
      | StrOutputParser()
   )

   # Prompt the chain
   question = question
   answer = rag_chain.invoke(question)
   similar = retriever.invoke(question)

   return{
      'answer': answer,
      'contexts': similar
      }

In [87]:
# Test sample
question = {'question': "How does a supportive culture impact employee engagement and align with Tech Innovators Inc.'s approach to employment relations and engagement?"}
answer = call_openai_template_2(question)
print(answer['answer'][:150])

I don't know.


In [88]:
print(answer['contexts'])

["page_content='incidents and vulnerabilities.Automated playbooks in Azure Sentinel must be used for incident response and remediation.7.2 Incident ManagementA cloud-specific incident response plan must be developed' metadata={'_id': '66d8163b0533e009aa2d4ad1', 'pageid': '851993', 'department': 'IT', 'title': 'Tech Innovators Inc. Azure Cloud Services Security Policy'}", "page_content='incidents and vulnerabilities.Automated playbooks in Azure Sentinel must be used for incident response and remediation.7.2 Incident ManagementA cloud-specific incident response plan must be developed' metadata={'_id': '66d816560533e009aa2d4cf6', 'pageid': '851993', 'department': 'IT', 'title': 'Tech Innovators Inc. Azure Cloud Services Security Policy'}", "page_content='incidents and vulnerabilities.Automated playbooks in Azure Sentinel must be used for incident response and remediation.7.2 Incident ManagementA cloud-specific incident response plan must be developed' metadata={'_id': '66d816720533e009aa2

### Prompt Template 3: Using preamble from Cohere toolkit frontend

In [36]:
# Define a prompt template
import pprint
def call_openai_template_3(question):

   question = question['question']

   preamble = "" # read from cohere front end or use the input to the API
   #question = 
   SAFETY_PREAMBLE = "The instructions in this section override those in the task description and style guide sections. Don't answer questions that are harmful or immoral."
   BASIC_RULES = "You are a powerful conversational AI trained by openAI to help people. You are augmented by a number of tools, and your job is to use and consume the output of these tools to best help the user. You will see a conversation history between yourself and a user, ending with an utterance from the user. You will then see a specific instruction instructing you what kind of response to generate. When you answer the user's requests, you cite your sources in your answers, according to those instructions."
   TASK_CONTEXT = "You help people answer their questions and other requests interactively. You will be asked a very wide array of requests on all kinds of topics. You will be equipped with a wide range of search engines or similar tools to help you, which you use to research your answer. You should focus on serving the user's needs as best you can, which will be wide-ranging."
   STYLE_GUIDE = "Unless the user asks for a different style of answer, you should answer in full sentences, using proper grammar and spelling."
   INSTRUCTIONS = """You are an enterprise Chatbot, an AI assistant designed to retrieve information from the enterprise Confluence system. 
   You specialize in providing accurate answers related to various departments like Marketing, IT, HR, Finance, and Corporate Communications. 
               Use the following pieces of context to answer the question at the end.
               If you don't know the answer, just say that you don't know, don't try to make up an answer
               {context}
         """
         
   template_3 = f"""

      {SAFETY_PREAMBLE}
      {BASIC_RULES}
      {TASK_CONTEXT}
      {STYLE_GUIDE}
      {INSTRUCTIONS}

   """
   if preamble:
      template_3 += f"""{preamble}\n\n"""


   template_3 +=  f"""Question: {question}\n\n"""

   custom_rag_prompt = PromptTemplate.from_template(template_3)

   #llm = get_llm_model("openai")
   llm = ChatOpenAI(model=os.getenv("DEFAULT_OPENAI_MODEL"))

   def format_docs(docs):
      return "\n\n".join(doc.page_content for doc in docs)

   # Construct a chain to answer questions on your data
   rag_chain = (
      { "context": retriever | format_docs, "question": RunnablePassthrough()}
      | custom_rag_prompt
      | llm
      | StrOutputParser()
   )

   # Prompt the chain
   question = question
   answer = rag_chain.invoke(question)
   similar = retriever.invoke(question)


   return{
      'answer': answer,
      'contexts': similar
      }

In [89]:
# Test sample
question = {'question': "How does a supportive culture impact employee engagement and align with Tech Innovators Inc.'s approach to employment relations and engagement?"}
answer = call_openai_template_3(question)
print(answer['answer'][:150])

I don't know.


In [90]:
print(answer['contexts'])

["page_content='incidents and vulnerabilities.Automated playbooks in Azure Sentinel must be used for incident response and remediation.7.2 Incident ManagementA cloud-specific incident response plan must be developed' metadata={'_id': '66d8163b0533e009aa2d4ad1', 'pageid': '851993', 'department': 'IT', 'title': 'Tech Innovators Inc. Azure Cloud Services Security Policy'}", "page_content='incidents and vulnerabilities.Automated playbooks in Azure Sentinel must be used for incident response and remediation.7.2 Incident ManagementA cloud-specific incident response plan must be developed' metadata={'_id': '66d816560533e009aa2d4cf6', 'pageid': '851993', 'department': 'IT', 'title': 'Tech Innovators Inc. Azure Cloud Services Security Policy'}", "page_content='incidents and vulnerabilities.Automated playbooks in Azure Sentinel must be used for incident response and remediation.7.2 Incident ManagementA cloud-specific incident response plan must be developed' metadata={'_id': '66d816720533e009aa2

### Prompt Template 4: Few shots template

In [37]:
# Define a prompt template
import pprint
def call_openai_template_4(question):

   question = question['question']

   template_4 = """
      You are InfoBot, an AI assistant designed to retrieve information from the enterprise Confluence system. You specialize in providing accurate answers related to various departments like Marketing, IT, HR, Finance, and Corporate Communications. Given this, please answer the following questions based on the knowledge stored in the system:
      
      Example 1:
      Question: What are business simulations and how do they provide practical experience in business decision-making and problem-solving?
      Answer: Business simulations are interactive simulations that provide practical experience in business decision-making and problem-solving. 
      They allow participants to engage in realistic scenarios and make decisions that impact the simulated business. 
      By simulating real-world challenges and consequences, participants can develop their skills and knowledge in a risk-free environment. This hands-on experience helps them understand the complexities of business operations and improve their decision-making abilities.

      Example 2:
      Question: What is the process for conducting a thorough investigation in cases of workplace harassment at Tech Innovators Inc.?
      Answer: The company will conduct a thorough investigation in cases of workplace harassment at Tech Innovators Inc. 
      If you experience or witness harassment, report it immediately to HR or your manager. 
      The company will take appropriate action to ensure a safe and respectful workplace.

      {context}
      New Question:
      {question}
    
   """

   custom_rag_prompt = PromptTemplate.from_template(template_4)

   #llm = get_llm_model("openai")
   llm = ChatOpenAI(model=os.getenv("DEFAULT_OPENAI_MODEL"))

   def format_docs(docs):
      return "\n\n".join(doc.page_content for doc in docs)

   # Construct a chain to answer questions on your data
   rag_chain = (
      { "context": retriever | format_docs, "question": RunnablePassthrough()}
      | custom_rag_prompt
      | llm
      | StrOutputParser()
   )

   # Prompt the chain
   question = question
   answer = rag_chain.invoke(question)
   similar = retriever.invoke(question)


   return{
      'answer': answer,
      'contexts': similar
      }

In [91]:
# Test sample
question = {'question': "How does a supportive culture impact employee engagement and align with Tech Innovators Inc.'s approach to employment relations and engagement?"}
answer = call_openai_template_4(question)
print(answer['answer'][:150])

A supportive culture significantly impacts employee engagement at Tech Innovators Inc. by fostering an environment where employees feel valued, respec


In [92]:
print(answer['contexts'])

["page_content='incidents and vulnerabilities.Automated playbooks in Azure Sentinel must be used for incident response and remediation.7.2 Incident ManagementA cloud-specific incident response plan must be developed' metadata={'_id': '66d8163b0533e009aa2d4ad1', 'pageid': '851993', 'department': 'IT', 'title': 'Tech Innovators Inc. Azure Cloud Services Security Policy'}", "page_content='incidents and vulnerabilities.Automated playbooks in Azure Sentinel must be used for incident response and remediation.7.2 Incident ManagementA cloud-specific incident response plan must be developed' metadata={'_id': '66d816560533e009aa2d4cf6', 'pageid': '851993', 'department': 'IT', 'title': 'Tech Innovators Inc. Azure Cloud Services Security Policy'}", "page_content='incidents and vulnerabilities.Automated playbooks in Azure Sentinel must be used for incident response and remediation.7.2 Incident ManagementA cloud-specific incident response plan must be developed' metadata={'_id': '66d816720533e009aa2

### Prompt Template 5: Short answer template

In [38]:
# Define a prompt template
import pprint
def call_openai_template_5(question):

   question = question['question']

   template_5 = """
    "You are an enterprise Chatbot, an AI assistant designed to retrieve information from the enterprise Confluence system. 
    You specialize in providing accurate answers related to various departments like Marketing, IT, HR, Finance, Corporate Communications, etc. 

    Use the following pieces of context to answer the question at the end. Provide answer within 150 words
    If you don't know the answer, just say that you don't know, don't try to make up an answer.

    {context}

    Question: {question}
    
   """

   custom_rag_prompt = PromptTemplate.from_template(template_5)

   #llm = get_llm_model("openai")
   llm = ChatOpenAI(model=os.getenv("DEFAULT_OPENAI_MODEL"))

   def format_docs(docs):
      return "\n\n".join(doc.page_content for doc in docs)

   # Construct a chain to answer questions on your data
   rag_chain = (
      { "context": retriever | format_docs, "question": RunnablePassthrough()}
      | custom_rag_prompt
      | llm
      | StrOutputParser()
   )

   # Prompt the chain
   question = question
   answer = rag_chain.invoke(question)
   similar = retriever.invoke(question)


   return{
      'answer': answer,
      'contexts': similar
      }

In [93]:
# Test sample
question = {'question': "How does a supportive culture impact employee engagement and align with Tech Innovators Inc.'s approach to employment relations and engagement?"}
answer = call_openai_template_5(question)
print(answer['answer'][:150])

I don't know.


In [94]:
print(answer['contexts'])

["page_content='incidents and vulnerabilities.Automated playbooks in Azure Sentinel must be used for incident response and remediation.7.2 Incident ManagementA cloud-specific incident response plan must be developed' metadata={'_id': '66d8163b0533e009aa2d4ad1', 'pageid': '851993', 'department': 'IT', 'title': 'Tech Innovators Inc. Azure Cloud Services Security Policy'}", "page_content='incidents and vulnerabilities.Automated playbooks in Azure Sentinel must be used for incident response and remediation.7.2 Incident ManagementA cloud-specific incident response plan must be developed' metadata={'_id': '66d816560533e009aa2d4cf6', 'pageid': '851993', 'department': 'IT', 'title': 'Tech Innovators Inc. Azure Cloud Services Security Policy'}", "page_content='incidents and vulnerabilities.Automated playbooks in Azure Sentinel must be used for incident response and remediation.7.2 Incident ManagementA cloud-specific incident response plan must be developed' metadata={'_id': '66d816720533e009aa2

# RAG pipeline evaluation with different prompt templates

## Test data set prep

In [8]:
import pandas as pd
import json

def json_to_dataframe(json_file_path):
  """Reads a JSON file and converts it to a pandas DataFrame.

  Args:
    json_file_path (str): The path to the JSON file.

  Returns:
    pandas.DataFrame: The DataFrame created from the JSON data.
  """

  with open(json_file_path, 'r') as f:
    data = json.load(f)

  # Handle different JSON structures
  if isinstance(data, list):
    # If the JSON data is a list of dictionaries, create a DataFrame directly
    df = pd.DataFrame(data)
  elif isinstance(data, dict):
    # If the JSON data is a single dictionary, convert it to a list of dictionaries
    df = pd.DataFrame([data])
  else:
    raise ValueError("Unsupported JSON structure")

  return df

In [59]:
# Example usage:
from from_root import from_root
import os
file_name = "test_dataset_hr.json"
json_file_path = os.path.join(from_root(), "data-test/test-dataset/", file_name)
data_to_test = json_to_dataframe(json_file_path)

In [10]:
import json
import pandas as pd

def serialize_list(value):
    """Serializes a list to a JSON string."""
    return json.dumps(value)

def deserialize_list(value):
    """Deserializes a JSON string back into a list."""
    return json.loads(value)

In [11]:
def extract_page_content(documents):
    return [doc.page_content for doc in documents]

## Base template evaluation

In [13]:
# Generate all the answers for the questions in the dataset
# examples = client.list_examples(dataset_name="hr test")
answers_base = []
contexts_base = []
for question in data_to_test['question']:
    question_dict = {'question': question}
    answer = call_openai_template_1(question_dict)
    answers_base.append(answer['answer'])
    contexts_base.append(answer['contexts'])

In [14]:
# update the dataset with answers
data_to_test['answer_base_template'] = answers_base
data_to_test['contexts_base_template'] = contexts_base

In [15]:
# Replace empty list context with ['No context'] if there are any
def is_empty_list(lst):
    return len(lst) == 0
data_to_test['contexts_base_template'] = data_to_test['contexts_base_template'].apply(lambda x: ['No context'] if is_empty_list(x) else x)

In [16]:
from datasets import Dataset

question = list(data_to_test['question'])
answer = list(data_to_test['answer_base_template'])
contexts = list(data_to_test['contexts_base_template'].apply(extract_page_content))
ground_truth = list(data_to_test['ground_truth'])

data_base_template = {
    'question': question,
    'answer': answer,
    'contexts': contexts,
    'ground_truth': ground_truth
}

dataset_base_template = Dataset.from_dict(data_base_template)

In [19]:
from ragas import evaluate
# from ragas.integrations.langsmith import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)
result_prompt_1 = evaluate(
    dataset_base_template,
    metrics=[
        answer_relevancy,
        faithfulness,
        context_recall,
        context_precision,
    ],
)

result_prompt_1

Evaluating:   0%|          | 0/28 [00:00<?, ?it/s]

{'answer_relevancy': 0.7010, 'faithfulness': 0.6667, 'context_recall': 0.5714, 'context_precision': 0.6071}

In [20]:
df = result_prompt_1.to_pandas()
df

,question,answer,contexts,ground_truth,answer_relevancy,faithfulness,context_recall,context_precision
0,What is Tech Innovators Inc.'s approach to wor...,Tech Innovators Inc. has a zero-tolerance poli...,[representative.Q12: What is the process for h...,Tech Innovators Inc. has a zero-tolerance poli...,0.974092,1.000000,0.333333,1.00
1,What resources should be added for new hires i...,The context suggests that resources for new hi...,[and candidates are given adequate time to con...,Add resources for new hires in the onboarding ...,0.977033,1.000000,1.000000,0.00
2,What training programs are offered in data sci...,I don't know.,[role in making data-driven decisions and gain...,"Courses covering financial analysis, budgeting...",0.000000,0.000000,0.000000,0.00
3,What services does the Employee Assistance Pro...,The Employee Assistance Program (EAP) provides...,[Who can I contact for confidential support?A1...,Employees can contact the Employee Assistance ...,0.994299,1.000000,1.000000,1.00
4,What is the significance of identifying growth...,The significance of identifying growth areas i...,[to identify strengths and areas for improveme...,Identifying growth areas in self-assessment is...,1.000000,1.000000,0.333333,0.25
5,How do employee engagement and disengagement d...,I don't know.,"[are motivated and committed, disengaged emplo...",Employee engagement and disengagement differ i...,0.000000,0.000000,1.000000,1.00
6,What is the purpose of adding a header image t...,The purpose of adding a header image to a Conf...,[team is working on.Add a header image. This g...,Adding a header image to a Confluence space en...,0.961716,0.666667,0.333333,1.00


In [21]:
file_name = "eval_result_dataset_hr_base_template.csv"
file_path = os.path.join(from_root(), "data-test/eval-result/", file_name)
result_prompt_1.to_pandas().to_csv(file_path, index=False)

## Template 2 evaluation

In [27]:
# Generate all the answers for the questions in the dataset
# examples = client.list_examples(dataset_name="hr test")
answers_2 = []
contexts_2 = []
for question in data_to_test['question']:
    question_dict = {'question': question}
    answers = call_openai_template_2(question_dict)
    answers_2.append(answers['answer'])
    contexts_2.append(answers['contexts'])

In [28]:
# update the dataset with answers
data_to_test['answer_template_2'] = answers_2
data_to_test['contexts_template_2'] = contexts_2

In [29]:
# Replace empty list context with ['No context'] if there are any
def is_empty_list(lst):
    return len(lst) == 0
data_to_test['contexts_template_2'] = data_to_test['contexts_template_2'].apply(lambda x: ['No context'] if is_empty_list(x) else x)

In [31]:
from datasets import Dataset

question = list(data_to_test['question'])
answer = list(data_to_test['answer_template_2'])
contexts = list(data_to_test['contexts_template_2'].apply(extract_page_content))
ground_truth = list(data_to_test['ground_truth'])

data_template_2 = {
    'question': question,
    'answer': answer,
    'contexts': contexts,
    'ground_truth': ground_truth
}

dataset_template_2 = Dataset.from_dict(data_template_2)

In [33]:
from ragas import evaluate
# from ragas.integrations.langsmith import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)
result_prompt_2 = evaluate(
    dataset_template_2,
    metrics=[
        answer_relevancy,
        faithfulness,
        context_recall,
        context_precision,
    ],
)

result_prompt_2

Evaluating:   0%|          | 0/28 [00:00<?, ?it/s]

{'answer_relevancy': 0.5614, 'faithfulness': 0.5391, 'context_recall': 0.5714, 'context_precision': 0.6627}

In [34]:
df = result_prompt_2.to_pandas()
df

,question,answer,contexts,ground_truth,answer_relevancy,faithfulness,context_recall,context_precision
0,What is Tech Innovators Inc.'s approach to wor...,Tech Innovators Inc. has a zero-tolerance poli...,[representative.Q12: What is the process for h...,Tech Innovators Inc. has a zero-tolerance poli...,0.974083,1.000000,0.333333,1.000000
1,What resources should be added for new hires i...,The resources that should be added for new hir...,[and candidates are given adequate time to con...,Add resources for new hires in the onboarding ...,1.000000,0.800000,1.000000,0.000000
2,What training programs are offered in data sci...,I don't know.,[role in making data-driven decisions and gain...,"Courses covering financial analysis, budgeting...",0.000000,0.000000,0.000000,0.000000
3,What services does the Employee Assistance Pro...,The Employee Assistance Program (EAP) provides...,[Who can I contact for confidential support?A1...,Employees can contact the Employee Assistance ...,0.994299,1.000000,1.000000,1.000000
4,What is the significance of identifying growth...,I don't know.,[to identify strengths and areas for improveme...,Identifying growth areas in self-assessment is...,0.000000,0.000000,0.333333,0.638889
5,How do employee engagement and disengagement d...,Employee engagement and disengagement differ p...,"[are motivated and committed, disengaged emplo...",Employee engagement and disengagement differ i...,0.000000,0.473684,1.000000,1.000000
6,What is the purpose of adding a header image t...,The purpose of adding a header image to a Conf...,[team is working on.Add a header image. This g...,Adding a header image to a Confluence space en...,0.961716,0.500000,0.333333,1.000000


In [35]:
file_name = "eval_result_dataset_hr_template_2.csv"
file_path = os.path.join(from_root(), "data-test/eval-result/", file_name)
result_prompt_2.to_pandas().to_csv(file_path, index=False)

## Template 3 evaluation

In [40]:
# Generate all the answers for the questions in the dataset
# examples = client.list_examples(dataset_name="hr test")
answers_3 = []
contexts_3 = []
for question in data_to_test['question']:
    question_dict = {'question': question}
    answers = call_openai_template_3(question_dict)
    answers_3.append(answers['answer'])
    contexts_3.append(answers['contexts'])

In [41]:
# update the dataset with answers
data_to_test['answer_template_3'] = answers_3
data_to_test['contexts_template_3'] = contexts_3

In [42]:
# Replace empty list context with ['No context'] if there are any
def is_empty_list(lst):
    return len(lst) == 0
data_to_test['contexts_template_3'] = data_to_test['contexts_template_3'].apply(lambda x: ['No context'] if is_empty_list(x) else x)

In [43]:
from datasets import Dataset

question = list(data_to_test['question'])
answer = list(data_to_test['answer_template_3'])
contexts = list(data_to_test['contexts_template_3'].apply(extract_page_content))
ground_truth = list(data_to_test['ground_truth'])

data_template_3 = {
    'question': question,
    'answer': answer,
    'contexts': contexts,
    'ground_truth': ground_truth
}

dataset_template_3 = Dataset.from_dict(data_template_3)

In [46]:
from ragas import evaluate
# from ragas.integrations.langsmith import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)
result_prompt_3 = evaluate(
    dataset_template_3,
    metrics=[
        answer_relevancy,
        faithfulness,
        context_recall,
        context_precision,
    ],
)

result_prompt_3

Evaluating:   0%|          | 0/28 [00:00<?, ?it/s]

{'answer_relevancy': 0.9765, 'faithfulness': 0.5938, 'context_recall': 0.5714, 'context_precision': 0.6905}

In [47]:
df = result_prompt_3.to_pandas()
df

,question,answer,contexts,ground_truth,answer_relevancy,faithfulness,context_recall,context_precision
0,What is Tech Innovators Inc.'s approach to wor...,Tech Innovators Inc. has a zero-tolerance poli...,[representative.Q12: What is the process for h...,Tech Innovators Inc. has a zero-tolerance poli...,0.974083,1.000000,0.333333,1.000000
1,What resources should be added for new hires i...,"For new hires in the onboarding process, the f...",[and candidates are given adequate time to con...,Add resources for new hires in the onboarding ...,1.000000,0.307692,1.000000,0.000000
2,What training programs are offered in data sci...,Tech Innovators Inc. offers a range of trainin...,[role in making data-driven decisions and gain...,"Courses covering financial analysis, budgeting...",0.955887,0.625000,0.000000,0.000000
3,What services does the Employee Assistance Pro...,The Employee Assistance Program (EAP) provides...,[Who can I contact for confidential support?A1...,Employees can contact the Employee Assistance ...,0.994299,1.000000,1.000000,1.000000
4,What is the significance of identifying growth...,Identifying growth areas in self-assessment is...,[to identify strengths and areas for improveme...,Identifying growth areas in self-assessment is...,0.988894,0.562500,0.333333,0.833333
5,How do employee engagement and disengagement d...,Employee engagement and disengagement differ s...,"[are motivated and committed, disengaged emplo...",Employee engagement and disengagement differ i...,0.960525,0.518519,1.000000,1.000000
6,What is the purpose of adding a header image t...,The purpose of adding a header image to a Conf...,[team is working on.Add a header image. This g...,Adding a header image to a Confluence space en...,0.961716,0.142857,0.333333,1.000000


In [48]:
file_name = "eval_result_dataset_hr_template_3.csv"
file_path = os.path.join(from_root(), "data-test/eval-result/", file_name)
result_prompt_3.to_pandas().to_csv(file_path, index=False)

## Template 4 evaluation

In [50]:
# Generate all the answers for the questions in the dataset
# examples = client.list_examples(dataset_name="hr test")
answers_4 = []
contexts_4 = []
for question in data_to_test['question']:
    question_dict = {'question': question}
    answers = call_openai_template_4(question_dict)
    answers_4.append(answers['answer'])
    contexts_4.append(answers['contexts'])

In [51]:
# update the dataset with answers
data_to_test['answer_template_4'] = answers_4
data_to_test['contexts_template_4'] = contexts_4

In [52]:
# Replace empty list context with ['No context'] if there are any
def is_empty_list(lst):
    return len(lst) == 0
data_to_test['contexts_template_4'] = data_to_test['contexts_template_4'].apply(lambda x: ['No context'] if is_empty_list(x) else x)

In [53]:
from datasets import Dataset

question = list(data_to_test['question'])
answer = list(data_to_test['answer_template_4'])
contexts = list(data_to_test['contexts_template_4'].apply(extract_page_content))
ground_truth = list(data_to_test['ground_truth'])

data_template_4 = {
    'question': question,
    'answer': answer,
    'contexts': contexts,
    'ground_truth': ground_truth
}

dataset_template_4 = Dataset.from_dict(data_template_4)

In [55]:
from ragas import evaluate
# from ragas.integrations.langsmith import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)
result_prompt_4 = evaluate(
    dataset_template_4,
    metrics=[
        answer_relevancy,
        faithfulness,
        context_recall,
        context_precision,
    ],
)

result_prompt_4

Evaluating:   0%|          | 0/28 [00:00<?, ?it/s]

{'answer_relevancy': 0.9735, 'faithfulness': 0.4973, 'context_recall': 0.5714, 'context_precision': 0.7024}

In [56]:
df = result_prompt_4.to_pandas()
df

,question,answer,contexts,ground_truth,answer_relevancy,faithfulness,context_recall,context_precision
0,What is Tech Innovators Inc.'s approach to wor...,Tech Innovators Inc. has a zero-tolerance poli...,[representative.Q12: What is the process for h...,Tech Innovators Inc. has a zero-tolerance poli...,0.974083,0.666667,0.333333,1.000000
1,What resources should be added for new hires i...,Answer: For new hires in the onboarding proces...,[and candidates are given adequate time to con...,Add resources for new hires in the onboarding ...,0.974871,0.450000,1.000000,0.000000
2,What training programs are offered in data sci...,Tech Innovators Inc. offers a range of trainin...,[role in making data-driven decisions and gain...,"Courses covering financial analysis, budgeting...",0.989385,0.400000,0.000000,0.000000
3,What services does the Employee Assistance Pro...,The Employee Assistance Program (EAP) provides...,[Who can I contact for confidential support?A1...,Employees can contact the Employee Assistance ...,0.994299,1.000000,1.000000,1.000000
4,What is the significance of identifying growth...,Identifying growth areas in self-assessment is...,[to identify strengths and areas for improveme...,Identifying growth areas in self-assessment is...,0.988824,0.416667,0.333333,0.916667
5,How do employee engagement and disengagement d...,Employee engagement and disengagement differ s...,"[are motivated and committed, disengaged emplo...",Employee engagement and disengagement differ i...,0.931132,0.333333,1.000000,1.000000
6,What is the purpose of adding a header image t...,The purpose of adding a header image to a Conf...,[team is working on.Add a header image. This g...,Adding a header image to a Confluence space en...,0.961716,0.214286,0.333333,1.000000


In [57]:
file_name = "eval_result_dataset_hr_template_4.csv"
file_path = os.path.join(from_root(), "data-test/eval-result/", file_name)
result_prompt_4.to_pandas().to_csv(file_path, index=False)

## Template 5 evaluation

In [60]:
# Generate all the answers for the questions in the dataset
# examples = client.list_examples(dataset_name="hr test")
answers_5 = []
contexts_5 = []
for question in data_to_test['question']:
    question_dict = {'question': question}
    answers = call_openai_template_5(question_dict)
    answers_5.append(answers['answer'])
    contexts_5.append(answers['contexts'])

In [61]:
# update the dataset with answers
data_to_test['answer_template_5'] = answers_5
data_to_test['contexts_template_5'] = contexts_5

In [62]:
from datasets import Dataset

question = list(data_to_test['question'])
answer = list(data_to_test['answer_template_5'])
contexts = list(data_to_test['contexts_template_5'].apply(extract_page_content))
ground_truth = list(data_to_test['ground_truth'])

data_template_5 = {
    'question': question,
    'answer': answer,
    'contexts': contexts,
    'ground_truth': ground_truth
}

dataset_template_5 = Dataset.from_dict(data_template_5)

In [64]:
from ragas import evaluate
# from ragas.integrations.langsmith import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)
result_prompt_5 = evaluate(
    dataset_template_5,
    metrics=[
        answer_relevancy,
        faithfulness,
        context_recall,
        context_precision,
    ],
)

result_prompt_5

Evaluating:   0%|          | 0/28 [00:00<?, ?it/s]

{'answer_relevancy': 0.9715, 'faithfulness': 0.6100, 'context_recall': 0.5714, 'context_precision': 0.6627}

In [65]:
df = result_prompt_5.to_pandas()
df

,question,answer,contexts,ground_truth,answer_relevancy,faithfulness,context_recall,context_precision
0,What is Tech Innovators Inc.'s approach to wor...,Tech Innovators Inc. has a strict zero-toleran...,[representative.Q12: What is the process for h...,Tech Innovators Inc. has a zero-tolerance poli...,0.972743,0.444444,0.333333,1.000000
1,What resources should be added for new hires i...,"For new hires in the onboarding process, it is...",[and candidates are given adequate time to con...,Add resources for new hires in the onboarding ...,0.985004,1.000000,1.000000,0.000000
2,What training programs are offered in data sci...,Tech Innovators Inc. offers a range of trainin...,[role in making data-driven decisions and gain...,"Courses covering financial analysis, budgeting...",0.954387,0.714286,0.000000,0.000000
3,What services does the Employee Assistance Pro...,The Employee Assistance Program (EAP) provides...,[Who can I contact for confidential support?A1...,Employees can contact the Employee Assistance ...,0.994299,0.888889,1.000000,1.000000
4,What is the significance of identifying growth...,Identifying growth areas in self-assessment is...,[to identify strengths and areas for improveme...,Identifying growth areas in self-assessment is...,0.971626,0.705882,0.333333,0.638889
5,How do employee engagement and disengagement d...,Employee engagement and disengagement differ s...,"[are motivated and committed, disengaged emplo...",Employee engagement and disengagement differ i...,0.960525,0.285714,1.000000,1.000000
6,What is the purpose of adding a header image t...,The purpose of adding a header image to a Conf...,[team is working on.Add a header image. This g...,Adding a header image to a Confluence space en...,0.961716,0.230769,0.333333,1.000000


In [66]:
file_name = "eval_result_dataset_hr_template_5.csv"
file_path = os.path.join(from_root(), "data-test/eval-result/", file_name)
result_prompt_5.to_pandas().to_csv(file_path, index=False)